In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [ ]:
# Load the dataset
file_path = 'arxiv_papers_enhanced_1.csv'
data = pd.read_csv(file_path)

print("Dataset Preview:")
print(data.head())

Dataset Preview:
                                               title  \
0  AI-generated Image Detection: Passive or Water...   
1  Generating 3D-Consistent Videos from Unposed I...   
2  HF-Diff: High-Frequency Perceptual Loss and Di...   
3  Promoting User Data Autonomy During the Dissol...   
4  Pushing the Limits of Sparsity: A Bag of Trick...   

                                             authors        published_date  \
0  Moyang Guo, Yuepeng Hu, Zhengyuan Jiang, Zeyu ...  2024-11-20T18:59:58Z   
1  Gene Chou, Kai Zhang, Sai Bi, Hao Tan, Zexiang...  2024-11-20T18:58:31Z   
2  Shoaib Meraj Sami, Md Mahedi Hasan, Jeremy Daw...  2024-11-20T18:56:24Z   
3                    Rushabh Solanki, Elliot Creager  2024-11-20T18:55:51Z   
4  Andy Li, Aiden Durrant, Milan Markovic, Lu Yin...  2024-11-20T18:54:53Z   

                                             summary           categories  \
0  While text-to-image models offer numerous bene...  cs.CR, cs.CV, cs.LG   
1  We address the probl

In [ ]:
# Gabungkan title dan summary untuk Representasi Teks
data['text'] = data['title'] + " " + data['summary']

In [ ]:
# Step 2: Hapus Kategori dengan Sedikit Sampel
threshold = 3  # Hapus kategori dengan kurang dari 3 sampel
data = data[data['main_category'].isin(
    data['main_category'].value_counts()[data['main_category'].value_counts() >= threshold].index
)]

# Periksa distribusi kategori setelah filtering
print("Dataset size:", data.shape)
print("\nCategory distribution:")
print(data['main_category'].value_counts())


Dataset size: (944, 8)

Category distribution:
main_category
cs.CV    282
cs.LG    160
cs.CL    120
cs.RO     70
cs.CR     43
cs.HC     42
cs.AI     33
cs.SE     25
cs.IR     25
cs.DC     17
cs.DS     17
cs.SD     14
cs.NI     12
cs.IT     12
cs.CY     11
cs.SI     10
cs.AR      6
cs.CE      6
cs.GT      6
cs.LO      6
cs.DB      5
cs.CC      5
cs.NE      4
cs.DL      4
cs.DM      3
cs.PL      3
cs.GR      3
Name: count, dtype: int64


In [ ]:
# Step 3: Split Data
X = data['text']
y = data['main_category']

# Split menjadi Training dan Testing Set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
# Step 4: TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [ ]:
# Step 5: Train Logistic Regression Model
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_tfidf, y_train)

# Predict
y_pred = model.predict(X_test_tfidf)


In [ ]:
# Evaluate on Testing Data
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("\nModel Performance on Testing Data:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Model Performance on Testing Data:
Accuracy: 0.52
Precision: 0.40
Recall: 0.52
F1 Score: 0.42


In [ ]:
# Evaluate on Training Data
y_train_pred = model.predict(X_train_tfidf)
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred, average='weighted', zero_division=0)
train_recall = recall_score(y_train, y_train_pred, average='weighted')
train_f1 = f1_score(y_train, y_train_pred, average='weighted')

print("\nModel Performance on Training Data:")
print(f"Accuracy: {train_accuracy:.2f}")
print(f"Precision: {train_precision:.2f}")
print(f"Recall: {train_recall:.2f}")
print(f"F1 Score: {train_f1:.2f}")


Model Performance on Training Data:
Accuracy: 0.68
Precision: 0.61
Recall: 0.68
F1 Score: 0.59


In [ ]:
# Step 6: Recommendation Function with Categories
def rekomendasikan_paper(input_pengguna, kategori_pilihan=None, jumlah_rekomendasi=5):
    """
    Merekomendasikan paper berdasarkan input pengguna (judul atau kata kunci) dan kategori yang dipilih.
    Args:
        input_pengguna (str): Judul atau kata kunci yang ingin dicari.
        kategori_pilihan (str): Kategori yang dipilih untuk menyaring rekomendasi (opsional).
        jumlah_rekomendasi (int): Jumlah rekomendasi yang ingin ditampilkan.
    Returns:
        DataFrame: Paper yang direkomendasikan dengan kolom judul, penulis, dan tautan.
    """
    # Filter berdasarkan kategori, jika ada
    if kategori_pilihan:
        data_filtered = data[data['categories'] == kategori_pilihan]
    else:
        data_filtered = data

    # Transformasi input pengguna ke vektor TF-IDF
    user_tfidf = tfidf_vectorizer.transform([input_pengguna])

    # Hitung skor kesamaan dengan semua paper dalam data yang difilter
    tfidf_matrix = tfidf_vectorizer.transform(data_filtered['text'])
    skor_kesamaan = cosine_similarity(user_tfidf, tfidf_matrix).flatten()

    # Dapatkan indeks paper dengan skor kesamaan tertinggi
    indeks_teratas = skor_kesamaan.argsort()[-jumlah_rekomendasi:][::-1]

    # Jika tidak ada rekomendasi yang ditemukan
    if skor_kesamaan[indeks_teratas[0]] == 0:
        return "Paper tidak ditemukan berdasarkan pencarian Anda."

    # Kembalikan rekomendasi paper dalam bentuk DataFrame
    rekomendasi = data_filtered.iloc[indeks_teratas][['title', 'authors', 'link']]
    return rekomendasi

In [ ]:
# Fungsi utama untuk sistem rekomendasi
def sistem_rekomendasi_paper():
    print("Selamat datang di Sistem Rekomendasi Paper!")
    input_awal = input("Masukkan judul atau kata kunci untuk mencari paper: ")

    # Langkah 1: Tampilkan rekomendasi pertama
    rekomendasi_awal = rekomendasikan_paper(input_awal)

    if isinstance(rekomendasi_awal, str):  # Cek apakah hasilnya berupa string (pesan error)
        print(rekomendasi_awal)
        return

    print("\nPaper yang Direkomendasikan:")
    print(rekomendasi_awal)

    # Langkah 2: Tanyakan kategori yang relevan
    print("\nPilih kategori yang relevan (misalnya: cs.CV, cs.LG, cs.AI, dll.):")

    # Daftar kategori beserta penjelasannya
    kategori_dict = {
        "cs.CV": "Computer Vision: Mempelajari teknik dan algoritma untuk memahami dan menganalisis gambar serta video.",
        "cs.LG": "Learning: Berfokus pada metode pembelajaran mesin dan statistik untuk memahami data.",
        "cs.RO": "Robotics: Mengembangkan robot yang dapat berinteraksi dengan dunia fisik.",
        "cs.HC": "Human-Computer Interaction: Memfokuskan pada interaksi antara manusia dan komputer.",
        "cs.AI": "Artificial Intelligence: Studi tentang mesin yang dapat meniru kecerdasan manusia.",
        "cs.SE": "Software Engineering: Mengembangkan software dengan pendekatan terstruktur dan efisien.",
        "cs.IR": "Information Retrieval: Mempelajari metode pencarian informasi yang efektif dari data besar.",
        "cs.DC": "Distributed Computing: Fokus pada sistem komputasi terdistribusi di banyak mesin.",
        "cs.DS": "Data Science: Menggunakan teknik statistik dan komputasi untuk menganalisis data besar.",
        "cs.SD": "Software Development: Pengembangan perangkat lunak dengan berbagai metode dan alat.",
        "cs.NI": "Network and Internet: Fokus pada sistem jaringan dan komunikasi di internet.",
        "cs.IT": "Information Technology: Mempelajari teknologi informasi dan pengelolaan sistem komputer.",
        "cs.CY": "Cybersecurity: Melindungi sistem komputer dari ancaman dan serangan dunia maya.",
        "cs.SI": "Software Infrastructure: Fokus pada pengembangan dan manajemen infrastruktur perangkat lunak.",
        "cs.AR": "Augmented Reality: Teknologi yang menggabungkan objek virtual dengan dunia nyata.",
        "cs.CE": "Computational Engineering: Aplikasi teknik komputasi dalam bidang teknik dan ilmu pengetahuan.",
        "cs.GT": "Graphics and Visualization: Fokus pada pembuatan grafik dan visualisasi data.",
        "cs.LO": "Logic: Studi tentang logika formal dan aplikasinya dalam komputasi.",
        "cs.DB": "Database: Mempelajari cara menyimpan, mengelola, dan mengakses data besar.",
        "cs.CC": "Cloud Computing: Menggunakan sumber daya komputasi secara online melalui cloud.",
        "cs.NE": "Neural Networks: Membangun sistem pembelajaran berbasis jaringan saraf tiruan.",
        "cs.DL": "Deep Learning: Pengembangan algoritma pembelajaran mesin yang lebih kompleks.",
        "cs.DM": "Data Mining: Menemukan pola atau informasi berharga dalam data besar.",
        "cs.PL": "Programming Languages: Mempelajari bahasa pemrograman dan paradigmannya.",
        "cs.GR": "Graphics: Fokus pada pembuatan gambar dan animasi digital."
    }

    # Tampilkan kategori dengan penjelasan
    for kategori, penjelasan in kategori_dict.items():
        print(f"{kategori}: {penjelasan}")

    kategori = input("\nMasukkan kategori yang relevan: ").strip()

    # Cek apakah kategori yang dimasukkan valid
    if kategori not in kategori_dict:
        print("Kategori tidak valid. Silakan pilih kategori yang tersedia.")
        return

    rekomendasi_bidang = rekomendasikan_paper(input_awal, kategori_pilihan=kategori)

    if isinstance(rekomendasi_bidang, str):  # Cek apakah hasilnya berupa string (pesan error)
        print(rekomendasi_bidang)
        return

    print(f"\nRekomendasi Paper untuk Kategori {kategori}:")
    print(rekomendasi_bidang)

    # Langkah 3: Pilih salah satu paper
    while True:
        try:
            pilihan = input("\nApakah paper sesuai dengan yang direkomendasikan (ya/tidak)? ").strip().lower()
            if pilihan == 'tidak':
                nomor_paper = int(input("Masukkan nomor (0 sampai 4) dari paper yang ingin Anda pilih: "))

                # Cek apakah nomor paper valid
                if nomor_paper < 0 or nomor_paper >= len(rekomendasi_bidang):
                    print("Nomor paper tidak valid. Silakan masukkan nomor yang benar.")
                    continue

                paper_dipilih = rekomendasi_bidang.iloc[nomor_paper]
                print(f"\nAnda memilih: {paper_dipilih['title']}")

                # Langkah 4: Masukkan paper baru untuk mengganti pilihan
                input_baru = input("\nMasukkan judul atau kata kunci baru untuk mengganti pilihan: ")
                rekomendasi_baru = rekomendasikan_paper(input_baru, kategori_pilihan=kategori)
                print("\nRekomendasi Baru Berdasarkan Input Anda:")
                print(rekomendasi_baru)
                break
            elif pilihan == 'ya':
                print("Terima kasih telah menggunakan Sistem Rekomendasi Paper!")
                break
            else:
                print("Pilihan tidak valid. Silakan masukkan 'ya' atau 'tidak'.")
        except ValueError:
            print("Input tidak valid. Harap masukkan nomor yang valid.")
            continue

# Jalankan sistem rekomendasi
sistem_rekomendasi_paper()


Selamat datang di Sistem Rekomendasi Paper!
Masukkan judul atau kata kunci untuk mencari paper: ai

Paper yang Direkomendasikan:
                                                 title  \
149  The Game-Theoretic Symbiosis of Trust and AI i...   
344  Human-AI Co-Creativity: Exploring Synergies Ac...   
955  Establishing Minimum Elements for Effective Vu...   
318  Declare and Justify: Explicit assumptions in A...   
619  From Score-Driven to Value-Sharing: Understand...   

                                               authors  \
149                             Yunfei Ge, Quanyan Zhu   
344                  Jennifer Haase, Sebastian Pokutta   
955  Mohamad Fazelnia, Sara Moshtari, Mehdi Mirakhorli   
318                      Peter Barnett, Lisa Thiergart   
619  Si Chen, Jingyi Xie, Ge Wang, Haizhou Wang, Ha...   

                                  link  
149  http://arxiv.org/abs/2411.12859v1  
344  http://arxiv.org/abs/2411.12527v1  
955  http://arxiv.org/abs/2411.11317v1  
318  http